### Detección de niveles operativos en base a retrocesos/avances de Fibonacci en SP500.

In [16]:
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates
from scipy.signal import argrelextrema
import numpy as np


data = pd.read_csv('data/Yahoo-SP500-2024.11.19.csv')
data = data.iloc[2:].rename(columns={'Price': 'datetime'})
data['datetime'] = pd.to_datetime(data['datetime'])
for col in ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']:
    data[col] = data[col].values.astype(float)
data.head(6)

datetime   Adj Close       Close        High         Low  \
2 1990-01-02 00:00:00+00:00  359.690002  359.690002  359.690002  351.980011   
3 1990-01-03 00:00:00+00:00  358.760010  358.760010  360.589996  357.890015   
4 1990-01-04 00:00:00+00:00  355.670013  355.670013  358.760010  352.890015   
5 1990-01-05 00:00:00+00:00  352.200012  352.200012  355.670013  351.350006   
6 1990-01-08 00:00:00+00:00  353.790009  353.790009  354.239990  350.540009   
7 1990-01-09 00:00:00+00:00  349.619995  349.619995  354.170013  349.609985   

         Open       Volume  
2  353.399994  162070000.0  
3  359.690002  192330000.0  
4  358.760010  177000000.0  
5  355.670013  158530000.0  
6  352.200012  140110000.0  
7  353.829987  155210000.0

In [17]:
data.isnull().sum()

datetime     0
Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64

In [18]:
fig = go.Figure(data=[go.Candlestick(x=data['datetime'],
                                       open=data['Open'],
                                       high=data['High'],
                                       low=data['Low'],
                                       close=data['Close'])])

fig.update_layout(title='S&P500 Candlestick Chart',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

fig.show()

En el caso mostrado, hay que hacer notar que la anomalía de mercado que supuso el COVID en 2020, anula toda acción del precio entre febrero de 2020 y noviembre de 2020. Esto hace que sean los 3591 puntos del 9 de noviembre de 2020 la primera vela sobre la que calcular avances y retrocesos. A partir de ese momento, el precio avanza firmemente hasta los 4808 del 27 de diciembre, momento en el que el precio se gira.

In [19]:
# Filter data for dates greater than or equal to November 9th 2020
filtered_data = data[data['datetime'] >= '2020-11-09']
filtered_data.set_index('datetime', inplace=True)
display(filtered_data.head())

fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                                       open=filtered_data['Open'],
                                       high=filtered_data['High'],
                                       low=filtered_data['Low'],
                                       close=filtered_data['Close'])])

fig.update_layout(title='S&P500 Candlestick Chart',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

fig.show()

Adj Close        Close         High          Low  \
datetime                                                                        
2020-11-09 00:00:00+00:00  3550.500000  3550.500000  3645.989990  3547.479980   
2020-11-10 00:00:00+00:00  3545.530029  3545.530029  3557.219971  3511.909912   
2020-11-11 00:00:00+00:00  3572.659912  3572.659912  3581.159912  3557.000000   
2020-11-12 00:00:00+00:00  3537.010010  3537.010010  3569.020020  3518.580078   
2020-11-13 00:00:00+00:00  3585.149902  3585.149902  3593.659912  3552.570068   

                                  Open        Volume  
datetime                                              
2020-11-09 00:00:00+00:00  3583.040039  8.570510e+09  
2020-11-10 00:00:00+00:00  3543.260010  6.037470e+09  
2020-11-11 00:00:00+00:00  3563.219971  4.635560e+09  
2020-11-12 00:00:00+00:00  3562.669922  4.909660e+09  
2020-11-13 00:00:00+00:00  3552.570068  4.719580e+09

#### Support & Resistance levels

In [20]:
def calculate_support_resistance(adj_close, order=5):
    """
    Calculate support and resistance levels based on local minima and maxima.
    
    :param adj_close: pandas Series of adjusted close prices
    :param order: Number of points to use for local extrema
    :return: support, resistance levels
    """
    # Find local minima (support) and maxima (resistance)
    local_minima = argrelextrema(adj_close.values, np.less, order=order)[0]
    local_maxima = argrelextrema(adj_close.values, np.greater, order=order)[0]
    
    # Get the support (minima) and resistance (maxima) prices
    support_levels = adj_close.iloc[local_minima]
    resistance_levels = adj_close.iloc[local_maxima]
    
    return support_levels, resistance_levels


adj_close = filtered_data["Adj Close"]
support, resistance = calculate_support_resistance(adj_close)


print("Support Levels:")
print(support)
print("\nResistance Levels:")
print(resistance)


Support Levels:
datetime
2020-11-12 00:00:00+00:00    3537.010010
2020-11-20 00:00:00+00:00    3557.540039
2020-12-14 00:00:00+00:00    3647.489990
2020-12-22 00:00:00+00:00    3687.260010
2021-01-04 00:00:00+00:00    3700.649902
                                ...     
2024-09-06 00:00:00+00:00    5408.419922
2024-10-07 00:00:00+00:00    5695.939941
2024-10-23 00:00:00+00:00    5797.419922
2024-10-31 00:00:00+00:00    5705.450195
2024-11-15 00:00:00+00:00    5870.620117
Name: Adj Close, Length: 66, dtype: float64

Resistance Levels:
datetime
2020-11-16 00:00:00+00:00    3626.909912
2020-12-08 00:00:00+00:00    3702.250000
2020-12-17 00:00:00+00:00    3722.479980
2021-01-08 00:00:00+00:00    3824.679932
2021-01-25 00:00:00+00:00    3855.360107
                                ...     
2024-07-31 00:00:00+00:00    5522.299805
2024-08-30 00:00:00+00:00    5648.399902
2024-09-30 00:00:00+00:00    5762.479980
2024-10-18 00:00:00+00:00    5864.669922
2024-11-11 00:00:00+00:00    6001.350098


### candlestick chart with support and resistance levels

In [21]:
adj_close = filtered_data['Adj Close'].dropna()

# Calculate Support and Resistance
def calculate_support_resistance(adj_close, order=5):
    """
    Calculate support and resistance levels based on local minima and maxima.
    """
    local_minima = argrelextrema(adj_close.values, np.less, order=order)[0]
    local_maxima = argrelextrema(adj_close.values, np.greater, order=order)[0]
    
    support_levels = adj_close.iloc[local_minima]
    resistance_levels = adj_close.iloc[local_maxima]
    
    return support_levels, resistance_levels

support, resistance = calculate_support_resistance(adj_close)

def plot_support_and_resistance(support, resistance):
    # Create a Candlestick chart using Plotly
    fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                    open=filtered_data['Open'],
                    high=filtered_data['High'],
                    low=filtered_data['Low'],
                    close=filtered_data['Adj Close'],
                    name='Candlesticks')])

    # Plot support and resistance levels
    fig.add_trace(go.Scatter(x=support.index, y=support.values, mode='markers', 
                             marker=dict(color='green', size=8, symbol='triangle-up'),
                             name='Support'))
    fig.add_trace(go.Scatter(x=resistance.index, y=resistance.values, mode='markers', 
                             marker=dict(color='red', size=8, symbol='triangle-down'),
                             name='Resistance'))

    # Customize layout
    fig.update_layout(title='Candlestick chart with Support and Resistance',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)

    # Show the plot
    fig.show()
plot_support_and_resistance(support, resistance)

The support and resistance points are too close together, this suggests that the function is too sensitive to small price changes. <b> UPDATE ... </b>

In [22]:
def calculate_support_resistance(adj_close, order=10, min_diff=0.02):
    """
    Calculate support and resistance levels based on local minima and maxima, 
    including the first point as support or resistance depending on its position.

    :param adj_close: pandas Series of adjusted close prices
    :param order: Number of points to use for local extrema
    :param min_diff: Minimum price difference between support/resistance levels
    :return: support, resistance levels
    """
    # Find local minima and maxima
    local_minima = argrelextrema(adj_close.values, np.less, order=order)[0]
    local_maxima = argrelextrema(adj_close.values, np.greater, order=order)[0]
    
    # Identify support and resistance from local minima and maxima
    support_levels = adj_close.iloc[local_minima]
    resistance_levels = adj_close.iloc[local_maxima]

    # Check if the first point is a local minimum or maximum
    first_price = adj_close.iloc[0]
    if first_price < adj_close.iloc[1:5].mean():  # First point is a local minimum
        support_levels = pd.concat([pd.Series([first_price], index=[adj_close.index[0]]), support_levels])
    elif first_price > adj_close.iloc[1:5].mean():  # First point is a local maximum
        resistance_levels = pd.concat([pd.Series([first_price], index=[adj_close.index[0]]), resistance_levels])

    # Filter out too-close support/resistance points
    support_levels = support_levels[support_levels.diff().abs() > min_diff]
    resistance_levels = resistance_levels[resistance_levels.diff().abs() > min_diff]
    
    return support_levels, resistance_levels

In [23]:
support, resistance = calculate_support_resistance(adj_close, order=12, min_diff=0.02)


plot_support_and_resistance(support, resistance)

Using the definition of calculate_support_resistance, the references are:

+ (2021-05-07 00:00:00+00:00 4232.600098)
+ (2020-11-12 00:00:00+00:00    3537.010010)

In [24]:
support_price = 3537.0100 
resistance_price = 4232.600098
support_price = support.iloc[0]
support_datetime = support.index[0]
resistance_price = resistance.iloc[0]
resistance_datetime = resistance.index[0]
current_datetime = max([support.index[0], resistance.index[0]])

def plot_fibo_retra(current_datetime, resistance_price, support_price):
    print(f'current_datetime: {current_datetime}')



    fib_levels = {
        "38.2%": resistance_price - (resistance_price - support_price) * 0.382,
        "50%": resistance_price - (resistance_price - support_price) * 0.5,
        "61.8%": resistance_price - (resistance_price - support_price) * 0.618,
    }

    fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                                         open=filtered_data['Open'],
                                         high=filtered_data['High'],
                                         low=filtered_data['Low'],
                                         close=filtered_data['Adj Close'],
                                         name='Candlesticks')])

    fig.add_trace(go.Scatter(x=[filtered_data.index[0], filtered_data.index[-1]], y=[support_price, support_price],
                             mode='lines', line=dict(color='green', dash='dash'), name='Support'))
    fig.add_trace(go.Scatter(x=[filtered_data.index[0], filtered_data.index[-1]], y=[resistance_price, resistance_price],
                             mode='lines', line=dict(color='red', dash='dash'), name='Resistance'))

    for label, level in fib_levels.items():
        fig.add_trace(go.Scatter(x=[filtered_data.index[0], filtered_data.index[-1]], y=[level, level],
                                 mode='lines', line=dict(color='blue', dash='dot'), name=f'Fibonacci {label}'))

    fig.update_layout(title='Candlestick chart with Fibonacci Retracements',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)

    # Show the plot
    fig.show()
    
plot_fibo_retra(current_datetime, resistance_price, support_price)


current_datetime: 2021-05-07 00:00:00+00:00


In [25]:
while True:
    support_tmp = support.loc[support.index > current_datetime]
    resistance_tmp = resistance.loc[resistance.index > current_datetime]
    
    if support_tmp.index[0] > resistance_tmp.index[0]:
        if support_tmp.iloc[0] >= support_price:
            pass
        else:
            # Update support
            print('Update support')
            support_price = support_tmp.iloc[0]
            current_datetime = support_tmp.index[0]
            support_datetime = support_tmp.index[0]
            break
    else:
        if resistance_tmp.iloc[0] <= resistance_price:
            pass
        else:
            # Update resistance
            print('Update resistance')
            resistance_price = resistance_tmp.iloc[0]
            resistance_datetime = resistance_tmp.index[0]
            current_datetime = resistance_tmp.index[0]
            break

print(f'current_datetime: {current_datetime}')
print(f'support_price: {support_price}')
print(f'resistance_price: {resistance_price}')


Update resistance
current_datetime: 2021-09-02 00:00:00+00:00
support_price: 3537.010009765625
resistance_price: 4536.9501953125


In [26]:
plot_fibo_retra(current_datetime, resistance_price, support_price)

current_datetime: 2021-09-02 00:00:00+00:00


In [27]:
def update_support_and_resistence(current_datetime, support, resistance, support_price, resistance_price,
                                 support_datetime, resistance_datetime):
    while True:
        support_tmp = support.loc[support.index > current_datetime]
        resistance_tmp = resistance.loc[resistance.index > current_datetime]

        if support_tmp.index[0] > resistance_tmp.index[0]:
            if support_tmp.iloc[0] >= support_price:
                pass
            else:
                # Update support
                print('Update support')
                support_price = support_tmp.iloc[0]
                current_datetime = support_tmp.index[0]
                support_datetime = support_tmp.index[0]
                break
        else:
            if resistance_tmp.iloc[0] <= resistance_price:
                pass
            else:
                # Update resistance
                print('Update resistance')
                resistance_price = resistance_tmp.iloc[0]
                resistance_datetime = resistance_tmp.index[0]
                current_datetime = resistance_tmp.index[0]
                break
    print(f'current_datetime: {current_datetime}')
    print(f'support_price: {support_price}')
    print(f'resistance_price: {resistance_price}')
    
    return current_datetime, support_price, resistance_price, support_datetime, resistance_datetime

current_datetime, support_price, resistance_price, support_datetime, resistance_datetime = \
                        update_support_and_resistence(current_datetime, 
                                        support, resistance, support_price, resistance_price,
                                                     support_datetime, resistance_datetime)
plot_fibo_retra(current_datetime, resistance_price, support_price)

Update resistance
current_datetime: 2021-11-18 00:00:00+00:00
support_price: 3537.010009765625
resistance_price: 4704.5400390625
current_datetime: 2021-11-18 00:00:00+00:00


In [28]:
current_datetime, support_price, resistance_price, support_datetime, resistance_datetime = \
                        update_support_and_resistence(current_datetime, 
                                        support, resistance, support_price, resistance_price,
                                                     support_datetime, resistance_datetime)
plot_fibo_retra(current_datetime, resistance_price, support_price)

Update resistance
current_datetime: 2022-01-03 00:00:00+00:00
support_price: 3537.010009765625
resistance_price: 4796.56005859375
current_datetime: 2022-01-03 00:00:00+00:00


##### Como detectar un cruce de fibo38?

In [29]:
fibo38 = resistance_price - (resistance_price - support_price) * 0.382
fibo68 =  resistance_price - (resistance_price - support_price) * 0.683

current_adj_close = adj_close.loc[adj_close.index == current_datetime].values[0]

assert current_adj_close > fibo38

support_tmp = support.loc[support.index > current_datetime]
resistance_tmp = resistance.loc[resistance.index > current_datetime]

adj_close_tmp = adj_close.loc[(adj_close.index >= current_datetime) & (adj_close.index >= support_tmp.index[0])
                             & (adj_close.index >= resistance_tmp.index[0])]
for idx, curr_adj_close in enumerate(adj_close_tmp):
    if curr_adj_close < fibo38:
        break
adj_close_cross = curr_adj_close
adj_close_datetime = adj_close_tmp.index[idx]

adj_close_datetime, adj_close_cross

(Timestamp('2022-04-22 00:00:00+0000', tz='UTC'), 4271.77978515625)

In [30]:
def plot_fibo_retra(current_datetime, resistance_price, support_price, cross_list=[]):
    print(f'current_datetime: {current_datetime}')

    # Fibonacci levels
    fib_levels = {
        "38.2%": resistance_price - (resistance_price - support_price) * 0.382,
        "50%": resistance_price - (resistance_price - support_price) * 0.5,
        "61.8%": resistance_price - (resistance_price - support_price) * 0.618,
    }

    # Create a Candlestick chart using Plotly
    fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                                         open=filtered_data['Open'],
                                         high=filtered_data['High'],
                                         low=filtered_data['Low'],
                                         close=filtered_data['Adj Close'],
                                         name='Candlesticks')])

    # Plot support and resistance levels
    fig.add_trace(go.Scatter(x=[filtered_data.index[0], filtered_data.index[-1]], y=[support_price, support_price],
                             mode='lines', line=dict(color='green', dash='dash'), name='Support'))
    fig.add_trace(go.Scatter(x=[filtered_data.index[0], filtered_data.index[-1]], y=[resistance_price, resistance_price],
                             mode='lines', line=dict(color='red', dash='dash'), name='Resistance'))

    # Plot Fibonacci levels
    for label, level in fib_levels.items():
        fig.add_trace(go.Scatter(x=[filtered_data.index[0], filtered_data.index[-1]], y=[level, level],
                                 mode='lines', line=dict(color='blue', dash='dot'), name=f'Fibonacci {label}'))

    # Add a blue cross at the specific point
    for blue_cross_x, blue_cross_y in cross_list:
        fig.add_trace(go.Scatter(x=[blue_cross_x], y=[blue_cross_y], mode='markers',
                                 marker=dict(color='blue', size=10, symbol='x'), name='Fibo38 Cross'))

    # Customize layout
    fig.update_layout(title='Candlestick chart with Fibonacci Retracements and Blue Cross',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)

    # Show the plot
    fig.show()

plot_fibo_retra(current_datetime, resistance_price, support_price, [(adj_close_datetime, adj_close_cross)])

current_datetime: 2022-01-03 00:00:00+00:00
